In [14]:
#Envorinment setting
#!pip install spacy
#!python3 -m spacy download en_core_web_sm
#!pip install torchtext==0.9.0
#!pip install torch==1.7.1
#!conda install pytorch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2 cudatoolkit=10.1 -c pytorch

     |████████████████████████████████| 6.5 MB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 12.7 MB 40.4 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 35.4 MB/s eta 0:00:01
     |████████████████████████████████| 835 kB 43.0 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.7 MB/s  eta 0:00:01
     |████████████████████████████████| 181 kB 39.7 MB/s eta 0:00:01
     |████████████████████████████████| 459 kB 73.4 MB/s eta 0:00:01
     |████████████████████████████████| 10.2 MB 26.5 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 7.7 MB/s  eta 0:00:01


In [1]:
from torchtext.legacy import data

In [2]:
# !pip install demoji
import nltk
from nltk.corpus import stopwords
# !pip install demoji
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /userhome/cs/u3591571/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /userhome/cs/u3591571/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /userhome/cs/u3591571/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /userhome/cs/u3591571/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /userhome/cs/u3591571/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-da

True

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# from wordcloud import WordCloud
import re
import unicodedata
import csv
import string
from textblob import TextBlob
import demoji

In [6]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
SEED = 1024
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

from torch.utils.data import TensorDataset, DataLoader, Dataset, RandomSampler, SequentialSampler
import torchtext
from torchtext import legacy
from torchtext import *

import transformers
from transformers import BertTokenizer, BertModel, BertConfig

import warnings
warnings.filterwarnings('ignore')
import operator

from sklearn.metrics import hamming_loss, jaccard_score, label_ranking_average_precision_score, f1_score,accuracy_score
from tqdm._tqdm_notebook import tqdm_notebook as tqdm

In [7]:
demoji.download_codes()

In [16]:
# !rm -rf "COVID19_sentinentanalysis"

In [59]:
!git clone https://github.com/xingxingaaa/COVID19_sentinentanalysis

fatal: destination path 'COVID19_sentinentanalysis' already exists and is not an empty directory.


In [136]:
senwave = pd.read_csv("labeledEn.csv")
print("Length of Senwave Dataset = {}".format(len(senwave)))

Length of Senwave Dataset = 10000


In [137]:
contractions = { 
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not",
"couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have",
"hasn't": "has not", "haven't": "have not", "he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'd've": "i would have",
"i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us",
"ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
"mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
"shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she shall / she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
"so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
"there'd've": "there would have", "there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
"they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
"we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
"who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have",
"y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
"you're": "you are", "you've": "you have"
}
contractionsWithAnotherInvertedComma = { 
"ain’t": "am not", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have", "’cause": "because", "could’ve": "could have", "couldn’t": "could not",
"couldn’t’ve": "could not have", "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have",
"hasn’t": "has not", "haven’t": "have not", "he’d": "he had", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "he’s": "he is",
"how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is", "i’d": "i would", "i’d’ve": "i would have",
"i’ll": "i will", "i’ll’ve": "i will have", "i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would",
"it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have", "it’s": "it is", "let’s": "let us",
"ma’am": "madam", "mayn’t": "may not", "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not",
"mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have",
"shan’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will",
"she’ll’ve": "she will have", "she’s": "she is", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have",
"so’ve": "so have", "so’s": "so is", "that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would",
"there’d’ve": "there would have", "there’s": "there is", "they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will",
"they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would",
"we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will", "who’ll’ve": "who will have",
"who’s": "who is", "who’ve": "who have", "why’s": "why is", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have",
"would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have",
"y’all’re": "you all are", "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have",
"you’re": "you are", "you’ve": "you have"
}

In [138]:
from preprocessing.preprocess import preprocess
pp_class = preprocess(senwave, contractions, contractionsWithAnotherInvertedComma) ## this is a class

In [139]:
senwave['Tweet'] = senwave['Tweet'].apply(lambda x : pp_class.preprocess_tweet(x))

In [140]:
senwave['Tweet'] = senwave['Tweet'].str.lower()
senwave.head(10)

,ID,Tweet,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Surprise,Joking
0,1245138832040890370,a glass of wine keeps the corona away drake ...,1,0,0,0,0,0,0,0,0,0,1
1,1245138859874234368,can anyone tell me if you took the flu shot la...,0,0,0,0,1,0,0,0,0,1,0
2,1245138869353353222,by the way producers send me beats im working ...,1,0,0,0,0,0,0,0,0,0,1
3,1245138873740648448,when someone you know apart of your family d...,0,0,0,0,0,1,0,0,0,0,0
4,1245138886172585989,dear soccer i really miss you please come ba...,0,0,0,0,0,1,1,0,0,0,0
5,1245138911195803649,new home remedy to treat coronavirus tested b...,1,0,0,0,0,0,0,1,0,0,1
6,1245138920934969344,when xavier wulf does an attack on titan tape ...,0,0,0,0,0,0,0,0,0,0,1
7,1245138945119223814,mouthwash is hand san for your mouth and i do ...,0,0,0,0,0,1,0,0,0,1,1
8,1245138953302409216,yes all of them n france 1 000 christians to...,0,0,0,0,0,0,1,0,1,1,0
9,1245138958222311431,update i destroyed the tire honestly if i get ...,0,0,0,1,0,1,0,0,0,0,0


In [141]:
from preprocessing import wordReplace
senwave['Tweet'] = senwave['Tweet'].apply(lambda x : wordReplace.bruteGen(x))

In [142]:
senwave.to_csv('senwave_preprocess.csv')

# Model for 11 labels

In [143]:
senwave = pd.read_csv('senwave_preprocess.csv')

In [152]:
senwave = senwave.drop(columns = ['Unnamed: 0'])

In [153]:
from sklearn.model_selection import train_test_split
sen_train, sen_test = train_test_split(senwave, train_size = 0.9, random_state = 42)

sen_train.to_csv("train.csv", index = False)
sen_test.to_csv("test.csv", index = False)

In [154]:
import spacy

# English pipeline optimized for CPU. Components: tok2vec, tagger, parser, senter, ner, attribute_ruler, lemmatizer.
spacy_en = spacy.load('en_core_web_sm')

# Spacy 会先将文档分解成句子，然后再tokenize。我们可以使用迭代来遍历整个文档
def tokenizer(tweet):
    tweet = re.sub(r'[\n]', ' ', tweet)
    return [tok.text for tok in spacy_en.tokenizer(tweet)] 

TWEET = torchtext.legacy.data.Field(sequential = True, lower = True, tokenize = tokenizer)  #tokenize 类型: function，作用: 文本处理，默认为str.split()
LABEL = torchtext.legacy.data.Field(sequential = False, use_vocab = False)

# 设置表头
dataFields = [("ID", None), ("Tweet", TWEET), ("Optimistic", LABEL), ("Thankful", LABEL),
              ("Empathetic", LABEL), ("Pessimistic", LABEL), ("Anxious", LABEL), ("Sad", LABEL),
              ("Annoyed", LABEL), ("Denial", LABEL), ("Official report", LABEL),
              ("Surprise", LABEL), ("Joking", LABEL)]
# 读取数据
train_dataset, test_dataset = torchtext.legacy.data.TabularDataset.splits(
    path = '', train = 'train.csv', test = 'test.csv', format = 'csv', fields = dataFields, skip_header = True
    )

In [155]:
# 构建词汇表
device = 'cuda' if torch.cuda.is_available() else 'cpu'
TWEET.build_vocab(train_dataset, vectors = 'glove.840B.300d') # get word embedding
# vectors, One of either the available pretrained vectors or custom pretrained vectors
# 然后构建语料库的 Vocabulary， 同时，加载预训练的 word-embedding(TEXT 字段的文本转化为数值)

In [158]:
# 修改TWEET.vocab
np.random.seed(1024)
for i in range(TWEET.vocab.vectors.shape[0]):
    vec = TWEET.vocab.vectors[i]
    if torch.sum(vec).item() == 0:
        a = np.random.uniform(-0.25, 0.25, 300)
        TWEET.vocab.vectors[i] = torch.from_numpy(a)

In [156]:
from torchtext.legacy.data import *

In [157]:
vocab = TWEET.vocab # 获得词汇表
BATCH_SIZE = 32
n_label= 11
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda')

# 文本批处理，即一批一批地读取数据
train_iter, test_iter = legacy.data.BucketIterator.splits(datasets = (train_dataset, test_dataset),
                                                   batch_size = BATCH_SIZE,
                                                   sort_key = lambda x : len(x.Tweet),
                                                   sort_within_batch = False,
                                                   repeat = False,
                                                   device = device)

In [159]:
batch = next(train_iter.__iter__()); batch


[torchtext.legacy.data.batch.Batch of size 32]
	[.Tweet]:[torch.cuda.LongTensor of size 28x32 (GPU 0)]
	[.Optimistic]:[torch.cuda.LongTensor of size 32 (GPU 0)]
	[.Thankful]:[torch.cuda.LongTensor of size 32 (GPU 0)]
	[.Empathetic]:[torch.cuda.LongTensor of size 32 (GPU 0)]
	[.Pessimistic]:[torch.cuda.LongTensor of size 32 (GPU 0)]
	[.Anxious]:[torch.cuda.LongTensor of size 32 (GPU 0)]
	[.Sad]:[torch.cuda.LongTensor of size 32 (GPU 0)]
	[.Annoyed]:[torch.cuda.LongTensor of size 32 (GPU 0)]
	[.Denial]:[torch.cuda.LongTensor of size 32 (GPU 0)]
	[.Official report]:[torch.cuda.LongTensor of size 32 (GPU 0)]
	[.Surprise]:[torch.cuda.LongTensor of size 32 (GPU 0)]
	[.Joking]:[torch.cuda.LongTensor of size 32 (GPU 0)]

In [160]:
#idea taken from http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/
# 对batch做个包装，方便调用
class BatchWrapper():
    def __init__(self, dl, x_var, y_vars):
        self.dl = dl
        self.x_var = x_var
        self.y_vars = y_vars

    def __iter__(self):
        for batch in self.dl:
            x = getattr(batch, self.x_var)
            if self.y_vars is not None:
                y = torch.cat([getattr(batch, feat).unsqueeze(1) for feat in self.y_vars], dim = 1).float()
            else:
                y = torch.zeros((1))
            yield(x, y)
    
    def __len__(self):
        return len(self.dl)

In [161]:
train_dl = BatchWrapper(train_iter, "Tweet", ['Optimistic', 'Thankful', 'Empathetic', 'Pessimistic', 
                                              'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Surprise', 'Joking'])
test_dl = BatchWrapper(test_iter, "Tweet", ['Optimistic', 'Thankful', 'Empathetic', 'Pessimistic', 
                                            'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Surprise', 'Joking'])

In [162]:
from torch import nn

In [163]:
class CustomLSTM(nn.Module):
    def __init__(self, embedding_dim, vocab, hidden_dim, output_dim, drop_prob, bidirectional = False, use_glove = True):
        super().__init__()
        self.embeddings = nn.Embedding(len(vocab), embedding_dim) # vocab_size词汇表大小， embedding_dim词嵌入维度
        
        if use_glove:
            self.embeddings.weight.data.copy_(vocab.vectors) # 如果使用glove
            self.embeddings.weight.requires_grad = False
            
        self.drop_prob = drop_prob # drop layer
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional = bidirectional, batch_first = True, num_layers = 2)
        
        if bidirectional is True:
            self.lin = nn.Linear(2*hidden_dim, 64)
        else:
            self.lin = nn.Linear(hidden_dim, 64) # 加入线性层
        
        self.fc = nn.Linear(64, output_dim) #这里是没有过sigmoid的
        
        self.dropout = nn.Dropout(p = drop_prob)
    
    def forward(self, sentence):
        #sentence = [max_len, batch_size]

        embed = self.embeddings(torch.transpose(sentence, 0, 1))
        #embed = [batch_size, max_len, embedding_dim]
        
        if self.drop_prob:
            embed = self.dropout(embed)
        
        lstm_out, (hidden, cell) = self.lstm(embed)
        #lstm_out = [batch_size, max_len, 2*hidden_dim if bidirectional else hidden_dim]
        #hidden = [num_layers, batch_size, hidden_dim]
        #cell = [num_layers, batch_size, hidden_dim]
        
        out = lstm_out[:,-1,:].squeeze()
        #out = [batch_size, 2*hidden_dim if bidirectional else hidden_dim]
        
        out = self.lin(out)
        #out = [batch_size, 64]

        outputs = self.fc(out)
        #outputs = [batch_size, output_dim]
        
        return outputs
    
def evaluation_metrics(actual_labels, pred_labels, threshold):
    int_pred_labels = pred_labels
    for i in range(len(pred_labels)):
        for j in range(11):
            if int_pred_labels[i][j] >= threshold: int_pred_labels[i][j] = 1
            else:
                int_pred_labels[i][j] = 0
    
    ham_loss = hamming_loss(actual_labels, int_pred_labels)
    accuracy_scores = accuracy_score(actual_labels, int_pred_labels)
    jacc_score = jaccard_score(actual_labels, int_pred_labels, average = 'samples')
    lrap = label_ranking_average_precision_score(actual_labels, pred_labels)
    f1_macro = f1_score(actual_labels, int_pred_labels, average = 'macro')
    f1_micro = f1_score(actual_labels, int_pred_labels, average = 'micro')

    return ham_loss,accuracy_scores, jacc_score, lrap, f1_macro, f1_micro

def train(model, loss_fn, optimizer, n_epochs, train_dl, threshold,test_dl):

    train_losses = []  
    accuracy = []
    hamming_losses = []
    jaccard_scores = []
    lraps = []  
    iter = 1

    for epoch in range(1, n_epochs+1):
        running_loss = 0.0
        pred_labels = []
        actual_labels = []
        model.train()
        for x, y in train_dl:
            # print(x.shape, y.shape) # torch.Size([27, 32]) torch.Size([32, 3])
            
            optimizer.zero_grad()

            preds = model(x) # 每一个类别输出了一个值
            # print('preds',preds)
            m = nn.Sigmoid()
            
            sig_preds = m(preds) # model输出值过了一个sigmoid, 在lstm当中是不会过sigmoid的吗
            # print('sig_preds',sig_preds)
            
            for tens in sig_preds:
                pred_labels.append(tens.cpu().detach().numpy())
            for tens in y:
                actual_labels.append(tens.cpu().detach().numpy())

            loss = loss_fn(preds, y) # preds是模型的输出值,y就是真实label
            
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * x.shape[0]

        ham_loss, accuracy_scores,jacc_score, lrap, f1_macro, f1_micro = evaluation_metrics(actual_labels, pred_labels, threshold)

        epoch_loss = running_loss / len(train_dataset)
        train_losses.append(epoch_loss)
        accuracy.append(accuracy_scores)
        hamming_losses.append(ham_loss)
        lraps.append(lrap)
        jaccard_scores.append(jacc_score)
        
        if iter % 2 == 0:
            print("Epoch: ", epoch)
            print("Binary Cross Entropy With Logits Loss: {:.4f}   ".format(epoch_loss),
                  "accuracy Score: {:.4f}".format(accuracy_scores))
            test_loss, accuracy_scores,test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
            print("TestBinary Cross Entropy With Logits Loss: {:.4f}   ".format(test_loss),
                 "accuracy Score: {:.4f}".format(accuracy_scores))
            print("\n")
        iter += 1

    return train_losses,accuracy, hamming_losses, jaccard_scores, lraps, f1_macro, f1_micro

def test(model, loss_fn, test_dl, threshold):
    running_loss = 0.0
    pred_labels = []
    actual_labels = []
    model.eval()
    for x, y in test_dl:
        #print(x.shape, y.shape)

        preds = model(x)

        m = nn.Sigmoid()
        sig_preds = m(preds)
        
        for tens in sig_preds:
            pred_labels.append(tens.cpu().detach().numpy())
        for tens in y:
            actual_labels.append(tens.cpu().detach().numpy())

        loss = loss_fn(preds, y)

        running_loss += loss.item() * x.shape[0]

    ham_loss, accuracy_scores, jacc_score, lrap, f1_macro, f1_micro = evaluation_metrics(actual_labels, pred_labels, threshold)

    test_loss = running_loss / len(test_dataset)
    return test_loss,accuracy_scores, ham_loss, jacc_score, lrap, f1_macro, f1_micro

### lstm with glove

In [164]:
# lstm with glove
learning_rates = [1e-3]
hidden_dims = [128]
thresholds= [0.5]
dropouts = [0.65]
n_epoch = 40
device = torch.device('cuda')
all_models = []
iter = 1

#number of iterations = 3*2*2 = 12 = number of models

for learning_rate in learning_rates:
    for hidden_dim in hidden_dims:
        for threshold in thresholds:
            for dropout in dropouts:
                model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 11, drop_prob = dropout)
                model = model.to(device)

                optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
                loss_fn = nn.BCEWithLogitsLoss()

                train_loss, accuracy,hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer,n_epoch, train_dl, threshold,test_dl)
                all_models.append(model)
                print("------------------------Final Train result-------------------")
                print("Train result")
                print("Learning Rate : {}, Hidden Dimension : {}, Threshold : {}, Dropout = {}".format(learning_rate, hidden_dim, threshold, dropout))
                print("Binary Cross Entropy With Logits Loss: {:.4f}".format(min(train_loss)))
                print("Hamming Loss : {:.4f}".format(min(hamm_loss)))
                print("Jaccard Score: {:.4f}".format(max(jacc_score)))
                print("accuracy Score: {:.4f}".format(max(accuracy)))
                print("Label Ranking Average Precision Score: {:.4f}".format(max(lrap)))
                print("F1 Macro Score: {:.4f}".format(f1_macro))
                print("F1 Micro Score: {:.4f}".format(f1_micro))
                print("\n")

                test_loss, accuracy_scores,test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
                print("TestBinary Cross Entropy With Logits Loss: {:.4f}".format(test_loss))
                print("Test Hamming Loss : {:.4f}".format(test_hamm_loss))
                print("Test Jaccard Score: {:.4f}".format(test_jacc_score))
                print("accuracy Score: {:.4f}".format(accuracy_scores))
                print("Test Label Ranking Average Precision Score: {:.4f}".format(test_lrap))
                print("Test F1 Macro Score: {:.4f}".format(test_f1_macro))
                print("Test F1 Micro Score: {:.4f}".format(test_f1_micro))
                print("\n")
                print("----------------------------------------------------------------")
                iter += 1

Epoch:  2
Binary Cross Entropy With Logits Loss: 0.3688    accuracy Score: 0.0787
TestBinary Cross Entropy With Logits Loss: 0.2491    accuracy Score: 0.0420


Epoch:  4
Binary Cross Entropy With Logits Loss: 0.3556    accuracy Score: 0.1111
TestBinary Cross Entropy With Logits Loss: 0.2455    accuracy Score: 0.1060


Epoch:  6
Binary Cross Entropy With Logits Loss: 0.3525    accuracy Score: 0.1150
TestBinary Cross Entropy With Logits Loss: 0.2429    accuracy Score: 0.1180


Epoch:  8
Binary Cross Entropy With Logits Loss: 0.3463    accuracy Score: 0.1183
TestBinary Cross Entropy With Logits Loss: 0.2375    accuracy Score: 0.1270


Epoch:  10
Binary Cross Entropy With Logits Loss: 0.3321    accuracy Score: 0.1413
TestBinary Cross Entropy With Logits Loss: 0.2315    accuracy Score: 0.1490


Epoch:  12
Binary Cross Entropy With Logits Loss: 0.3234    accuracy Score: 0.1552
TestBinary Cross Entropy With Logits Loss: 0.2263    accuracy Score: 0.1740


Epoch:  14
Binary Cross Entropy With L

### lstm without glove

In [165]:
# lstm without glove
learning_rates = [1e-3]
hidden_dims = [128]
thresholds = [0.5]
dropouts = [0.65]
n_epoch = 40
device = torch.device('cuda')
all_models = []
iter = 1

#number of iterations = 3*2*2 = 12 = number of models

for learning_rate in learning_rates:
    for hidden_dim in hidden_dims:
        for threshold in thresholds:
            for dropout in dropouts:
                model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 11, drop_prob = dropout,use_glove = False)
                model = model.to(device)

                optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
                loss_fn = nn.BCEWithLogitsLoss()

                train_loss, accuracy,hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer,n_epoch, train_dl, threshold,test_dl)
                all_models.append(model)
                print("------------------------Final Train result-------------------")
                print("Train result")
                print("Learning Rate : {}, Hidden Dimension : {}, Threshold : {}, Dropout = {}".format(learning_rate, hidden_dim, threshold, dropout))
                print("Binary Cross Entropy With Logits Loss: {:.4f}".format(min(train_loss)))
                print("Hamming Loss : {:.4f}".format(min(hamm_loss)))
                print("Jaccard Score: {:.4f}".format(max(jacc_score)))
                print("accuracy Score: {:.4f}".format(max(accuracy)))
                print("Label Ranking Average Precision Score: {:.4f}".format(max(lrap)))
                print("F1 Macro Score: {:.4f}".format(f1_macro))
                print("F1 Micro Score: {:.4f}".format(f1_micro))
                print("\n")

                test_loss, accuracy_scores,test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
                print("TestBinary Cross Entropy With Logits Loss: {:.4f}".format(test_loss))
                print("Test Hamming Loss : {:.4f}".format(test_hamm_loss))
                print("Test Jaccard Score: {:.4f}".format(test_jacc_score))
                print("accuracy Score: {:.4f}".format(accuracy_scores))
                print("Test Label Ranking Average Precision Score: {:.4f}".format(test_lrap))
                print("Test F1 Macro Score: {:.4f}".format(test_f1_macro))
                print("Test F1 Micro Score: {:.4f}".format(test_f1_micro))
                print("\n")
                print("----------------------------------------------------------------")
                iter += 1

Epoch:  2
Binary Cross Entropy With Logits Loss: 0.3863    accuracy Score: 0.0136
TestBinary Cross Entropy With Logits Loss: 0.2637    accuracy Score: 0.0730


Epoch:  4
Binary Cross Entropy With Logits Loss: 0.3652    accuracy Score: 0.1003
TestBinary Cross Entropy With Logits Loss: 0.2590    accuracy Score: 0.1220


Epoch:  6
Binary Cross Entropy With Logits Loss: 0.3562    accuracy Score: 0.1064
TestBinary Cross Entropy With Logits Loss: 0.2604    accuracy Score: 0.1170


Epoch:  8
Binary Cross Entropy With Logits Loss: 0.3509    accuracy Score: 0.1119
TestBinary Cross Entropy With Logits Loss: 0.2575    accuracy Score: 0.1210


Epoch:  10
Binary Cross Entropy With Logits Loss: 0.3444    accuracy Score: 0.1146
TestBinary Cross Entropy With Logits Loss: 0.2585    accuracy Score: 0.1210


Epoch:  12
Binary Cross Entropy With Logits Loss: 0.3413    accuracy Score: 0.1228
TestBinary Cross Entropy With Logits Loss: 0.2574    accuracy Score: 0.1280


Epoch:  14
Binary Cross Entropy With L

### bi- lstm with glove

In [169]:
# bi- lstm with glove
learning_rates = [1e-3]
hidden_dims = [128]
thresholds = [0.5]
dropouts = [0.65]
n_epoch = 25
device = torch.device('cuda')
all_models = []
iter = 1

#number of iterations = 3*2*2 = 12 = number of models

for learning_rate in learning_rates:
    for hidden_dim in hidden_dims:
        for threshold in thresholds:
            for dropout in dropouts:
                model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 11, drop_prob = dropout,
                                   bidirectional = True, use_glove = True)
                model = model.to(device)

                optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
                loss_fn = nn.BCEWithLogitsLoss()

                train_loss, accuracy,hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer,n_epoch, train_dl, threshold,test_dl)
                all_models.append(model)
                print("------------------------Final Train result-------------------")
                print("Train result")
                print("Learning Rate : {}, Hidden Dimension : {}, Threshold : {}, Dropout = {}".format(learning_rate, hidden_dim, threshold, dropout))
                print("Binary Cross Entropy With Logits Loss: {:.4f}".format(min(train_loss)))
                print("Hamming Loss : {:.4f}".format(min(hamm_loss)))
                print("Jaccard Score: {:.4f}".format(max(jacc_score)))
                print("accuracy Score: {:.4f}".format(max(accuracy)))
                print("Label Ranking Average Precision Score: {:.4f}".format(max(lrap)))
                print("F1 Macro Score: {:.4f}".format(f1_macro))
                print("F1 Micro Score: {:.4f}".format(f1_micro))
                print("\n")

                test_loss, accuracy_scores,test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
                print("TestBinary Cross Entropy With Logits Loss: {:.4f}".format(test_loss))
                print("Test Hamming Loss : {:.4f}".format(test_hamm_loss))
                print("Test Jaccard Score: {:.4f}".format(test_jacc_score))
                print("accuracy Score: {:.4f}".format(accuracy_scores))
                print("Test Label Ranking Average Precision Score: {:.4f}".format(test_lrap))
                print("Test F1 Macro Score: {:.4f}".format(test_f1_macro))
                print("Test F1 Micro Score: {:.4f}".format(test_f1_micro))
                print("\n")
                print("----------------------------------------------------------------")
                iter += 1

Epoch:  2
Binary Cross Entropy With Logits Loss: 0.3704    accuracy Score: 0.0846
TestBinary Cross Entropy With Logits Loss: 0.2469    accuracy Score: 0.1100


Epoch:  4
Binary Cross Entropy With Logits Loss: 0.3561    accuracy Score: 0.1079
TestBinary Cross Entropy With Logits Loss: 0.2448    accuracy Score: 0.1200


Epoch:  6
Binary Cross Entropy With Logits Loss: 0.3511    accuracy Score: 0.1147
TestBinary Cross Entropy With Logits Loss: 0.2422    accuracy Score: 0.1010


Epoch:  8
Binary Cross Entropy With Logits Loss: 0.3363    accuracy Score: 0.1402
TestBinary Cross Entropy With Logits Loss: 0.2324    accuracy Score: 0.1580


Epoch:  10
Binary Cross Entropy With Logits Loss: 0.3272    accuracy Score: 0.1566
TestBinary Cross Entropy With Logits Loss: 0.2322    accuracy Score: 0.1510


Epoch:  12
Binary Cross Entropy With Logits Loss: 0.3205    accuracy Score: 0.1650
TestBinary Cross Entropy With Logits Loss: 0.2289    accuracy Score: 0.1660


Epoch:  14
Binary Cross Entropy With L

### bi-lstm without glove

In [167]:
# bi- lstm without glove
learning_rates = [1e-3]
hidden_dims = [128]
thresholds = [0.5]
dropouts = [0.65]
n_epoch = 45
device = torch.device('cuda')
all_models = []
iter = 1

#number of iterations = 3*2*2 = 12 = number of models

for learning_rate in learning_rates:
    for hidden_dim in hidden_dims:
        for threshold in thresholds:
            for dropout in dropouts:
                model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 11, drop_prob = dropout,
                                   bidirectional = True, use_glove = False)
                model = model.to(device)

                optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
                loss_fn = nn.BCEWithLogitsLoss()

                train_loss, accuracy,hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer,n_epoch, train_dl, threshold,test_dl)
                all_models.append(model)
                print("------------------------Final Train result-------------------")
                print("Train result")
                print("Learning Rate : {}, Hidden Dimension : {}, Threshold : {}, Dropout = {}".format(learning_rate, hidden_dim, threshold, dropout))
                print("Binary Cross Entropy With Logits Loss: {:.4f}".format(min(train_loss)))
                print("Hamming Loss : {:.4f}".format(min(hamm_loss)))
                print("Jaccard Score: {:.4f}".format(max(jacc_score)))
                print("accuracy Score: {:.4f}".format(max(accuracy)))
                print("Label Ranking Average Precision Score: {:.4f}".format(max(lrap)))
                print("F1 Macro Score: {:.4f}".format(f1_macro))
                print("F1 Micro Score: {:.4f}".format(f1_micro))
                print("\n")

                test_loss, accuracy_scores,test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
                print("TestBinary Cross Entropy With Logits Loss: {:.4f}".format(test_loss))
                print("Test Hamming Loss : {:.4f}".format(test_hamm_loss))
                print("Test Jaccard Score: {:.4f}".format(test_jacc_score))
                print("accuracy Score: {:.4f}".format(accuracy_scores))
                print("Test Label Ranking Average Precision Score: {:.4f}".format(test_lrap))
                print("Test F1 Macro Score: {:.4f}".format(test_f1_macro))
                print("Test F1 Micro Score: {:.4f}".format(test_f1_micro))
                print("\n")
                print("----------------------------------------------------------------")
                iter += 1

Epoch:  2
Binary Cross Entropy With Logits Loss: 0.3891    accuracy Score: 0.0060
TestBinary Cross Entropy With Logits Loss: 0.2697    accuracy Score: 0.0500


Epoch:  4
Binary Cross Entropy With Logits Loss: 0.3637    accuracy Score: 0.0976
TestBinary Cross Entropy With Logits Loss: 0.2532    accuracy Score: 0.1130


Epoch:  6
Binary Cross Entropy With Logits Loss: 0.3540    accuracy Score: 0.1096
TestBinary Cross Entropy With Logits Loss: 0.2502    accuracy Score: 0.1090


Epoch:  8
Binary Cross Entropy With Logits Loss: 0.3493    accuracy Score: 0.1127
TestBinary Cross Entropy With Logits Loss: 0.2486    accuracy Score: 0.1240


Epoch:  10
Binary Cross Entropy With Logits Loss: 0.3444    accuracy Score: 0.1177
TestBinary Cross Entropy With Logits Loss: 0.2492    accuracy Score: 0.1260


Epoch:  12
Binary Cross Entropy With Logits Loss: 0.3377    accuracy Score: 0.1232
TestBinary Cross Entropy With Logits Loss: 0.2470    accuracy Score: 0.1330


Epoch:  14
Binary Cross Entropy With L

## Model 3 labels

In [115]:
senwave = pd.read_csv('senwave_preprocess_version2.csv')

In [116]:
senwave = senwave[['ID','Tweet','Positive','Negative','Neutural']]

In [117]:
from sklearn.model_selection import train_test_split
sen_train, sen_test = train_test_split(senwave, train_size = 0.9, random_state = 1024)

sen_train.to_csv("train.csv", index = False)
sen_test.to_csv("test.csv", index = False)

In [118]:
import spacy
spacy_en = spacy.load('en_core_web_sm')

def tokenizer(tweet):
    tweet = re.sub(r'[\n]', ' ', tweet)
    return [tok.text for tok in spacy_en.tokenizer(tweet)]

TWEET = torchtext.legacy.data.Field(sequential = True, lower = True, tokenize = tokenizer)
LABEL = torchtext.legacy.data.Field(sequential = False, use_vocab = False)

dataFields = [("ID", None), ("Tweet", TWEET), ("Positive", LABEL), ("Negative", LABEL),
              ("Neutural", LABEL)]

train_dataset, test_dataset = torchtext.legacy.data.TabularDataset.splits(
    path = '', train = 'train.csv', test = 'test.csv', format = 'csv', fields = dataFields, skip_header = True
    )

In [119]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
TWEET.build_vocab(train_dataset, vectors = 'glove.840B.300d') # get word embedding

In [120]:
from torchtext.legacy.data import *

In [121]:
vocab = TWEET.vocab # data 
BATCH_SIZE = 32
n_label= 3
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda')

train_iter, test_iter = legacy.data.BucketIterator.splits(datasets = (train_dataset, test_dataset),
                                                   batch_size = BATCH_SIZE,
                                                   sort_key = lambda x : len(x.Tweet),
                                                   sort_within_batch = False,
                                                   repeat = False,
                                                   device = device)

In [122]:
# 修改TWEET.vocab
np.random.seed(1024)
for i in range(TWEET.vocab.vectors.shape[0]):
    vec = TWEET.vocab.vectors[i]
    if torch.sum(vec).item() == 0:
        a = np.random.uniform(-0.25, 0.25, 300)
        TWEET.vocab.vectors[i] = torch.from_numpy(a)

In [123]:
batch = next(train_iter.__iter__()); batch


[torchtext.legacy.data.batch.Batch of size 32]
	[.Tweet]:[torch.cuda.LongTensor of size 31x32 (GPU 0)]
	[.Positive]:[torch.cuda.LongTensor of size 32 (GPU 0)]
	[.Negative]:[torch.cuda.LongTensor of size 32 (GPU 0)]
	[.Neutural]:[torch.cuda.LongTensor of size 32 (GPU 0)]

In [124]:
#idea taken from http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/
class BatchWrapper():
    def __init__(self, dl, x_var, y_vars):
        self.dl = dl
        self.x_var = x_var
        self.y_vars = y_vars

    def __iter__(self):
        for batch in self.dl:
            x = getattr(batch, self.x_var)
            if self.y_vars is not None:
                y = torch.cat([getattr(batch, feat).unsqueeze(1) for feat in self.y_vars], dim = 1).float()
            else:
                y = torch.zeros((1))
            yield(x, y)
    
    def __len__(self):
        return len(self.dl)

In [125]:
train_dl = BatchWrapper(train_iter, "Tweet", ['Positive','Negative','Neutural'])
test_dl = BatchWrapper(test_iter, "Tweet", ['Positive','Negative','Neutural'])

In [126]:
from torch import nn

In [129]:
class CustomLSTM(nn.Module):
    def __init__(self, embedding_dim, vocab, hidden_dim, output_dim, drop_prob, bidirectional = False, use_glove = True):
        super().__init__()
        self.embeddings = nn.Embedding(len(vocab), embedding_dim) # vocab是数据，使用的这种embedding的方式
        
        if use_glove:
            self.embeddings.weight.data.copy_(vocab.vectors) # 如果使用glove，这里其实不是很懂呢
            self.embeddings.weight.requires_grad = False
            
        self.drop_prob = drop_prob # drop layer
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional = bidirectional, batch_first = True, num_layers = 2)
        
        if bidirectional is True:
            self.lin = nn.Linear(2*hidden_dim, 64)
        else:
            self.lin = nn.Linear(hidden_dim, 64)
        
        self.fc = nn.Linear(64, output_dim)
        
        self.dropout = nn.Dropout(p = drop_prob)
    
    def forward(self, sentence):
        #sentence = [max_len, batch_size]

        embed = self.embeddings(torch.transpose(sentence, 0, 1))
        #embed = [batch_size, max_len, embedding_dim]
        
        if self.drop_prob:
            embed = self.dropout(embed)
        
        lstm_out, (hidden, cell) = self.lstm(embed)
        #lstm_out = [batch_size, max_len, 2*hidden_dim if bidirectional else hidden_dim]
        #hidden = [num_layers, batch_size, hidden_dim]
        #cell = [num_layers, batch_size, hidden_dim]
        
        out = lstm_out[:,-1,:].squeeze()
        #out = [batch_size, 2*hidden_dim if bidirectional else hidden_dim]
        
        out = self.lin(out)
        #out = [batch_size, 64]

        outputs = self.fc(out)
        #outputs = [batch_size, output_dim]
        
        return outputs
    
def evaluation_metrics(actual_labels, pred_labels, threshold):
    
#     int_pred_labels = pred_labels
#     for i in range(len(pred_labels)):
#          for j in range(3):
#             if int_pred_labels[i][j] >= threshold: 
#                 int_pred_labels[i][j] = 1
#             else:
#                 int_pred_labels[i][j] = 0
    
    int_pred_labels = pred_labels
    for i in range(len(pred_labels)):
        max_index = np.argmax(pred_labels[i])
        for j in range(3):
            if j == max_index:
                int_pred_labels[i][j] = 1
            else:
                int_pred_labels[i][j] = 0
        
    ham_loss = hamming_loss(actual_labels, int_pred_labels)
    accuracy_scores = accuracy_score(actual_labels, int_pred_labels)
    jacc_score = jaccard_score(actual_labels, int_pred_labels, average = 'samples')
    lrap = label_ranking_average_precision_score(actual_labels, pred_labels)
    f1_macro = f1_score(actual_labels, int_pred_labels, average = 'macro')
    f1_micro = f1_score(actual_labels, int_pred_labels, average = 'micro')

    return ham_loss,accuracy_scores, jacc_score, lrap, f1_macro, f1_micro

def train(model, loss_fn, optimizer, n_epochs, train_dl, threshold,test_dl):

    train_losses = []  
    accuracy = []
    hamming_losses = []
    jaccard_scores = []
    lraps = []  
    iter = 1

    for epoch in range(1, n_epochs+1):
        running_loss = 0.0
        pred_labels = []
        actual_labels = []
        model.train()
        for x, y in train_dl:
            #print(x.shape, y.shape)
            optimizer.zero_grad()

            preds = model(x)

            m =  nn.Softmax(dim=1) # 在这里取softmax
            
            sig_preds = m(preds) 
            for tens in sig_preds:
                pred_labels.append(tens.cpu().detach().numpy())
            for tens in y:
                actual_labels.append(tens.cpu().detach().numpy())
            #return preds,y
            
            y_loss = y.argmax(dim = 1)
            loss = loss_fn(preds, y_loss)
            

            # loss = loss_fn(preds, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * x.shape[0]
        # print("pred_labels",pred_labels)
        # print('actual_labels',actual_labels)
        ham_loss, accuracy_scores,jacc_score, lrap, f1_macro, f1_micro = evaluation_metrics(actual_labels, pred_labels, threshold)

        epoch_loss = running_loss / len(train_dataset)
        train_losses.append(epoch_loss)
        accuracy.append(accuracy_scores)
        hamming_losses.append(ham_loss)
        lraps.append(lrap)
        jaccard_scores.append(jacc_score)
        
        if iter % 2 == 0:
            print("Epoch: ", epoch)
            print("Binary Cross Entropy With Logits Loss: {:.4f}   ".format(epoch_loss),
                  "accuracy Score: {:.4f}".format(accuracy_scores))
            test_loss, accuracy_scores,test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
            print("TestBinary Cross Entropy With Logits Loss: {:.4f}   ".format(test_loss),
                 "accuracy Score: {:.4f}".format(accuracy_scores))
            print("\n")
        iter += 1

    return train_losses,accuracy, hamming_losses, jaccard_scores, lraps, f1_macro, f1_micro

def test(model, loss_fn, test_dl, threshold):
    running_loss = 0.0
    pred_labels = []
    actual_labels = []
    model.eval()
    for x, y in test_dl:
        #print(x.shape, y.shape)

        preds = model(x) 

        m = nn.Softmax(dim=1) 
        sig_preds = m(preds)
        
        for tens in sig_preds:
            pred_labels.append(tens.cpu().detach().numpy())
        for tens in y:
            actual_labels.append(tens.cpu().detach().numpy())

        y_loss = y.argmax(dim = 1)
        loss = loss_fn(preds, y_loss)

        running_loss += loss.item() * x.shape[0]

    ham_loss, accuracy_scores, jacc_score, lrap, f1_macro, f1_micro = evaluation_metrics(actual_labels, pred_labels, threshold)

    test_loss = running_loss / len(test_dataset)
    return test_loss,accuracy_scores, ham_loss, jacc_score, lrap, f1_macro, f1_micro


def confusion(model, loss_fn, test_dl, threshold):
    running_loss = 0.0
    pred_labels = []
    actual_labels = []
    model.eval()
    for x, y in test_dl:
        #print(x.shape, y.shape)
        preds = model(x)
        m = nn.Softmax(dim=1) 
        sig_preds = m(preds)
        for tens in sig_preds:
            pred_labels.append(tens.cpu().detach().numpy())
        for tens in y:
            actual_labels.append(tens.cpu().detach().numpy())
        loss = loss_fn(preds, y)
        running_loss += loss.item() * x.shape[0]
    
    pred_final = []
    for i in range(len(pred_labels)):
        pred_final.append(np.argmax(pred_labels[i]))
        
    actual_final = []
    for i in range(len(actual_labels)):
        actual_final.append(np.argmax(actual_labels[i]))
    
    result = confusion_matrix(actual_final, pred_final)

    return result

## Lstm with Glove

In [131]:
learning_rates = [1e-3]
hidden_dims = [128]
thresholds = [0.5]
dropouts = [0.65]
n_epoch = 10
device = torch.device('cuda')
all_models = []
iter = 1

#number of iterations = 3*2*2 = 12 = number of models

for learning_rate in learning_rates:
    for hidden_dim in hidden_dims:
        for threshold in thresholds:
            for dropout in dropouts:
                model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 3, drop_prob = dropout,
                                   bidirectional = False, use_glove = True)
                model = model.to(device)

                optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
                loss_fn = nn.CrossEntropyLoss()
                
                # preds,y = train(model, loss_fn, optimizer,n_epoch, train_dl, threshold,test_dl)
                
                train_loss, accuracy,hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer,n_epoch, train_dl, threshold,test_dl)
                all_models.append(model)
                print("------------------------Final Train result-------------------")
                print("Train result")
                print("Learning Rate : {}, Hidden Dimension : {}, Threshold : {}, Dropout = {}".format(learning_rate, hidden_dim, threshold, dropout))
                print("Binary Cross Entropy With Logits Loss: {:.4f}".format(min(train_loss)))
                print("Hamming Loss : {:.4f}".format(min(hamm_loss)))
                print("Jaccard Score: {:.4f}".format(max(jacc_score)))
                print("accuracy Score: {:.4f}".format(max(accuracy)))
                print("Label Ranking Average Precision Score: {:.4f}".format(max(lrap)))
                print("F1 Macro Score: {:.4f}".format(f1_macro))
                print("F1 Micro Score: {:.4f}".format(f1_micro))
                print("\n")

                test_loss, accuracy_scores,test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
                print("TestBinary Cross Entropy With Logits Loss: {:.4f}".format(test_loss))
                print("Test Hamming Loss : {:.4f}".format(test_hamm_loss))
                print("Test Jaccard Score: {:.4f}".format(test_jacc_score))
                print("accuracy Score: {:.4f}".format(accuracy_scores))
                print("Test Label Ranking Average Precision Score: {:.4f}".format(test_lrap))
                print("Test F1 Macro Score: {:.4f}".format(test_f1_macro))
                print("Test F1 Micro Score: {:.4f}".format(test_f1_micro))
                print("\n")
                print("----------------------------------------------------------------")
                iter += 1

Epoch:  2
Binary Cross Entropy With Logits Loss: 0.8714    accuracy Score: 0.5789
TestBinary Cross Entropy With Logits Loss: 0.5753    accuracy Score: 0.5885


Epoch:  4
Binary Cross Entropy With Logits Loss: 0.7849    accuracy Score: 0.6195
TestBinary Cross Entropy With Logits Loss: 0.5179    accuracy Score: 0.6254


Epoch:  6
Binary Cross Entropy With Logits Loss: 0.7546    accuracy Score: 0.6340
TestBinary Cross Entropy With Logits Loss: 0.5004    accuracy Score: 0.6298


Epoch:  8
Binary Cross Entropy With Logits Loss: 0.7301    accuracy Score: 0.6470
TestBinary Cross Entropy With Logits Loss: 0.5112    accuracy Score: 0.6232


Epoch:  10
Binary Cross Entropy With Logits Loss: 0.7125    accuracy Score: 0.6556
TestBinary Cross Entropy With Logits Loss: 0.4868    accuracy Score: 0.6287


------------------------Final Train result-------------------
Train result
Learning Rate : 0.001, Hidden Dimension : 128, Threshold : 0.5, Dropout = 0.65
Binary Cross Entropy With Logits Loss: 0.7125

### LSTM with out GLove

In [132]:
learning_rates = [1e-3]
hidden_dims = [128]
thresholds = [0.5]
dropouts = [0.65]
n_epoch = 10
device = torch.device('cuda')
all_models = []
iter = 1

#number of iterations = 3*2*2 = 12 = number of models

for learning_rate in learning_rates:
    for hidden_dim in hidden_dims:
        for threshold in thresholds:
            for dropout in dropouts:
                model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 3, drop_prob = dropout,
                                   bidirectional = False, use_glove = False)
                model = model.to(device)

                optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
                loss_fn = nn.CrossEntropyLoss()
                
                # preds,y = train(model, loss_fn, optimizer,n_epoch, train_dl, threshold,test_dl)
                
                train_loss, accuracy,hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer,n_epoch, train_dl, threshold,test_dl)
                all_models.append(model)
                print("------------------------Final Train result-------------------")
                print("Train result")
                print("Learning Rate : {}, Hidden Dimension : {}, Threshold : {}, Dropout = {}".format(learning_rate, hidden_dim, threshold, dropout))
                print("Binary Cross Entropy With Logits Loss: {:.4f}".format(min(train_loss)))
                print("Hamming Loss : {:.4f}".format(min(hamm_loss)))
                print("Jaccard Score: {:.4f}".format(max(jacc_score)))
                print("accuracy Score: {:.4f}".format(max(accuracy)))
                print("Label Ranking Average Precision Score: {:.4f}".format(max(lrap)))
                print("F1 Macro Score: {:.4f}".format(f1_macro))
                print("F1 Micro Score: {:.4f}".format(f1_micro))
                print("\n")

                test_loss, accuracy_scores,test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
                print("TestBinary Cross Entropy With Logits Loss: {:.4f}".format(test_loss))
                print("Test Hamming Loss : {:.4f}".format(test_hamm_loss))
                print("Test Jaccard Score: {:.4f}".format(test_jacc_score))
                print("accuracy Score: {:.4f}".format(accuracy_scores))
                print("Test Label Ranking Average Precision Score: {:.4f}".format(test_lrap))
                print("Test F1 Macro Score: {:.4f}".format(test_f1_macro))
                print("Test F1 Micro Score: {:.4f}".format(test_f1_micro))
                print("\n")
                print("----------------------------------------------------------------")
                iter += 1

Epoch:  2
Binary Cross Entropy With Logits Loss: 0.8884    accuracy Score: 0.5630
TestBinary Cross Entropy With Logits Loss: 0.5849    accuracy Score: 0.5852


Epoch:  4
Binary Cross Entropy With Logits Loss: 0.8420    accuracy Score: 0.5823
TestBinary Cross Entropy With Logits Loss: 0.5782    accuracy Score: 0.5928


Epoch:  6
Binary Cross Entropy With Logits Loss: 0.8001    accuracy Score: 0.6073
TestBinary Cross Entropy With Logits Loss: 0.6458    accuracy Score: 0.6048


Epoch:  8
Binary Cross Entropy With Logits Loss: 0.7527    accuracy Score: 0.6288
TestBinary Cross Entropy With Logits Loss: 0.5801    accuracy Score: 0.6145


Epoch:  10
Binary Cross Entropy With Logits Loss: 0.7062    accuracy Score: 0.6506
TestBinary Cross Entropy With Logits Loss: 0.6261    accuracy Score: 0.6102


------------------------Final Train result-------------------
Train result
Learning Rate : 0.001, Hidden Dimension : 128, Threshold : 0.5, Dropout = 0.65
Binary Cross Entropy With Logits Loss: 0.7062

### Bi LSTM with GLove

In [133]:
learning_rates = [1e-3]
hidden_dims = [128]
thresholds = [0.5]
dropouts = [0.65]
n_epoch = 10
device = torch.device('cuda')
all_models = []
iter = 1

#number of iterations = 3*2*2 = 12 = number of models

for learning_rate in learning_rates:
    for hidden_dim in hidden_dims:
        for threshold in thresholds:
            for dropout in dropouts:
                model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 3, drop_prob = dropout,
                                   bidirectional = True, use_glove = True)
                model = model.to(device)

                optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
                loss_fn = nn.CrossEntropyLoss()
                
                # preds,y = train(model, loss_fn, optimizer,n_epoch, train_dl, threshold,test_dl)
                
                train_loss, accuracy,hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer,n_epoch, train_dl, threshold,test_dl)
                all_models.append(model)
                print("------------------------Final Train result-------------------")
                print("Train result")
                print("Learning Rate : {}, Hidden Dimension : {}, Threshold : {}, Dropout = {}".format(learning_rate, hidden_dim, threshold, dropout))
                print("Binary Cross Entropy With Logits Loss: {:.4f}".format(min(train_loss)))
                print("Hamming Loss : {:.4f}".format(min(hamm_loss)))
                print("Jaccard Score: {:.4f}".format(max(jacc_score)))
                print("accuracy Score: {:.4f}".format(max(accuracy)))
                print("Label Ranking Average Precision Score: {:.4f}".format(max(lrap)))
                print("F1 Macro Score: {:.4f}".format(f1_macro))
                print("F1 Micro Score: {:.4f}".format(f1_micro))
                print("\n")

                test_loss, accuracy_scores,test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
                print("TestBinary Cross Entropy With Logits Loss: {:.4f}".format(test_loss))
                print("Test Hamming Loss : {:.4f}".format(test_hamm_loss))
                print("Test Jaccard Score: {:.4f}".format(test_jacc_score))
                print("accuracy Score: {:.4f}".format(accuracy_scores))
                print("Test Label Ranking Average Precision Score: {:.4f}".format(test_lrap))
                print("Test F1 Macro Score: {:.4f}".format(test_f1_macro))
                print("Test F1 Micro Score: {:.4f}".format(test_f1_micro))
                print("\n")
                print("----------------------------------------------------------------")
                iter += 1

Epoch:  2
Binary Cross Entropy With Logits Loss: 0.8780    accuracy Score: 0.5636
TestBinary Cross Entropy With Logits Loss: 0.5665    accuracy Score: 0.5668


Epoch:  4
Binary Cross Entropy With Logits Loss: 0.8116    accuracy Score: 0.5898
TestBinary Cross Entropy With Logits Loss: 0.5191    accuracy Score: 0.6156


Epoch:  6
Binary Cross Entropy With Logits Loss: 0.7543    accuracy Score: 0.6367
TestBinary Cross Entropy With Logits Loss: 0.4928    accuracy Score: 0.6428


Epoch:  8
Binary Cross Entropy With Logits Loss: 0.7293    accuracy Score: 0.6493
TestBinary Cross Entropy With Logits Loss: 0.5188    accuracy Score: 0.6265


Epoch:  10
Binary Cross Entropy With Logits Loss: 0.6999    accuracy Score: 0.6591
TestBinary Cross Entropy With Logits Loss: 0.5021    accuracy Score: 0.6352


------------------------Final Train result-------------------
Train result
Learning Rate : 0.001, Hidden Dimension : 128, Threshold : 0.5, Dropout = 0.65
Binary Cross Entropy With Logits Loss: 0.6999

### Bi lstm without glove

In [134]:
learning_rates = [1e-3]
hidden_dims = [128]
thresholds = [0.5]
dropouts = [0.65]
n_epoch = 10
device = torch.device('cuda')
all_models = []
iter = 1

#number of iterations = 3*2*2 = 12 = number of models

for learning_rate in learning_rates:
    for hidden_dim in hidden_dims:
        for threshold in thresholds:
            for dropout in dropouts:
                model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 3, drop_prob = dropout,
                                   bidirectional = True, use_glove = False)
                model = model.to(device)

                optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
                loss_fn = nn.CrossEntropyLoss()
                
                # preds,y = train(model, loss_fn, optimizer,n_epoch, train_dl, threshold,test_dl)
                
                train_loss, accuracy,hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer,n_epoch, train_dl, threshold,test_dl)
                all_models.append(model)
                print("------------------------Final Train result-------------------")
                print("Train result")
                print("Learning Rate : {}, Hidden Dimension : {}, Threshold : {}, Dropout = {}".format(learning_rate, hidden_dim, threshold, dropout))
                print("Binary Cross Entropy With Logits Loss: {:.4f}".format(min(train_loss)))
                print("Hamming Loss : {:.4f}".format(min(hamm_loss)))
                print("Jaccard Score: {:.4f}".format(max(jacc_score)))
                print("accuracy Score: {:.4f}".format(max(accuracy)))
                print("Label Ranking Average Precision Score: {:.4f}".format(max(lrap)))
                print("F1 Macro Score: {:.4f}".format(f1_macro))
                print("F1 Micro Score: {:.4f}".format(f1_micro))
                print("\n")

                test_loss, accuracy_scores,test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
                print("TestBinary Cross Entropy With Logits Loss: {:.4f}".format(test_loss))
                print("Test Hamming Loss : {:.4f}".format(test_hamm_loss))
                print("Test Jaccard Score: {:.4f}".format(test_jacc_score))
                print("accuracy Score: {:.4f}".format(accuracy_scores))
                print("Test Label Ranking Average Precision Score: {:.4f}".format(test_lrap))
                print("Test F1 Macro Score: {:.4f}".format(test_f1_macro))
                print("Test F1 Micro Score: {:.4f}".format(test_f1_micro))
                print("\n")
                print("----------------------------------------------------------------")
                iter += 1

Epoch:  2
Binary Cross Entropy With Logits Loss: 0.8874    accuracy Score: 0.5628
TestBinary Cross Entropy With Logits Loss: 0.5908    accuracy Score: 0.5646


Epoch:  4
Binary Cross Entropy With Logits Loss: 0.8460    accuracy Score: 0.5711
TestBinary Cross Entropy With Logits Loss: 0.5931    accuracy Score: 0.5494


Epoch:  6
Binary Cross Entropy With Logits Loss: 0.7949    accuracy Score: 0.5996
TestBinary Cross Entropy With Logits Loss: 0.6042    accuracy Score: 0.5733


Epoch:  8
Binary Cross Entropy With Logits Loss: 0.7585    accuracy Score: 0.6235
TestBinary Cross Entropy With Logits Loss: 0.5996    accuracy Score: 0.5896


Epoch:  10
Binary Cross Entropy With Logits Loss: 0.7062    accuracy Score: 0.6540
TestBinary Cross Entropy With Logits Loss: 0.6061    accuracy Score: 0.5668


------------------------Final Train result-------------------
Train result
Learning Rate : 0.001, Hidden Dimension : 128, Threshold : 0.5, Dropout = 0.65
Binary Cross Entropy With Logits Loss: 0.7062

## Bert 

In [24]:
df = senwave.drop(['ID'], axis = 1)
df['list'] = df[df.columns[1:12]].values.tolist()
new_df = df[['Tweet', 'list']].copy()
new_df.head()

,Tweet,list
0,a glass of wine keeps the corona away drake ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
1,can anyone tell me if you took the flu shot la...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]"
2,by the way producers send me beats im working ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
3,when someone you know apart of your family d...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
4,dear soccer i really miss you please come ba...,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]"


In [25]:
MAX_LEN = 200 #based on length of tweets
TRAIN_BATCH_SIZE = 1
VALID_BATCH_SIZE = 1
EPOCHS = 4
LEARNING_RATE = 1e-05 #tried 1e-03, 1e-04, 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [26]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.tweet = dataframe['Tweet']
        self.targets = self.dataframe.list
        self.max_len = max_len

    def __len__(self):
        return len(self.tweet)
    
    def __getitem__(self, index):
        tweet = str(self.tweet[index])
        tweet = " ".join(tweet.split())

        inputs = self.tokenizer.encode_plus(
            tweet,
            None,
            add_special_tokens = True,
            max_length = self.max_len,
            pad_to_max_length = True,
            return_token_type_ids = True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']

        return {
            'ids' : torch.tensor(ids, dtype = torch.long),
            'mask' : torch.tensor(mask, dtype = torch.long),
            'token_type_ids' : torch.tensor(token_type_ids, dtype = torch.long),
            'targets' : torch.tensor(self.targets[index], dtype = torch.float)
        }

In [27]:
train_dataset = sen_train.drop(['ID'], axis = 1)
train_dataset['list'] = train_dataset[train_dataset.columns[1:12]].values.tolist()
train_df = train_dataset[['Tweet', 'list']].copy()
train_df = train_df.reset_index(drop = True)

test_dataset = sen_test.drop(['ID'], axis = 1)
test_dataset['list'] = test_dataset[test_dataset.columns[1:12]].values.tolist()
test_df = test_dataset[['Tweet', 'list']].copy()
test_df = test_df.reset_index(drop = True)

In [28]:
training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

In [29]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [30]:
class BERT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.layer2 = torch.nn.Dropout(0.3)
        self.layer3 = torch.nn.Linear(768, 11)

    def forward(self, ids, mask, token_type_ids, return_dict = False):
        unw, out_1 = self.layer1(ids, attention_mask = mask, token_type_ids = token_type_ids)[0], self.layer1(ids, attention_mask = mask, token_type_ids = token_type_ids)[1]
        out_2 = self.layer2(out_1)
        out_final = self.layer3(out_2)
        return out_final

model = BERT()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT(
  (layer1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

In [31]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params = model.parameters(), lr = LEARNING_RATE)

In [32]:
def train(epoch):
    model.train()
    total_loss = 0

    for unw, data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids, return_dict = False)
        loss = loss_fn(outputs, targets)
        total_loss += loss.item()

        if unw % 2000 == 0:
            print(f'Iter : {unw+1}, Epoch: {epoch+1}, Loss: {total_loss/(unw+1)}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [33]:
EPOCHS = 4
for epoch in range(EPOCHS):
    train(epoch)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Iter : 1, Epoch: 1, Loss: 0.6962275505065918
Iter : 2001, Epoch: 1, Loss: 0.4257115384508883
Iter : 4001, Epoch: 1, Loss: 0.4031909396054893
Iter : 6001, Epoch: 1, Loss: 0.38547705694563766
Iter : 8001, Epoch: 1, Loss: 0.37646117098345455
Iter : 1, Epoch: 2, Loss: 0.28627488017082214
Iter : 2001, Epoch: 2, Loss: 0.30361873152731417
Iter : 4001, Epoch: 2, Loss: 0.3006949994673031
Iter : 6001, Epoch: 2, Loss: 0.30041549270738266
Iter : 8001, Epoch: 2, Loss: 0.30066742445892236
Iter : 1, Epoch: 3, Loss: 0.23145930469036102
Iter : 2001, Epoch: 3, Loss: 0.2394077192074147
Iter : 4001, Epoch: 3, Loss: 0.23735791795938962
Iter : 6001, Epoch: 3, Loss: 0.23757051523148498
Iter : 8001, Epoch: 3, Loss: 0.24030982081623184
Iter : 1, Epoch: 4, Loss: 0.2412097454071045
Iter : 2001, Epoch: 4, Loss: 0.17873933605723147
Iter : 4001, Epoch: 4, Loss: 0.1792770810318206
Iter : 6001, Epoch: 4, Loss: 0.18177160203448808
Iter : 8001, Epoch: 4, Loss: 0.1835075834197529


In [34]:
def valid():
    model.eval()
    req_targets = []
    req_outputs = []
    valid_loss = 0

    with torch.no_grad():
        for unw, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            loss = loss_fn(outputs, targets)
            valid_loss += loss.item()

            req_targets.extend(targets.cpu().detach().numpy().tolist())
            req_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    valid_loss /= len(testing_loader)
    return req_outputs, req_targets, valid_loss

In [35]:
from sklearn import metrics
outputs, targets, valid_loss = valid()

In [36]:
outputs = np.array(outputs)
targets = np.array(targets)

In [37]:
int_outputs = np.zeros((outputs.shape[0], outputs.shape[1]))

for row in range(outputs.shape[0]):
    for col in range(outputs.shape[1]):
        if outputs[row][col] >= 0.5: int_outputs[row][col] = 1

In [38]:
bert_ham_loss = hamming_loss(targets, int_outputs)
bert_jacc_score = jaccard_score(targets, int_outputs, average = 'samples')
bert_lrap = label_ranking_average_precision_score(targets, outputs)
bert_f1_macro = f1_score(targets, int_outputs, average = 'macro')
bert_f1_micro = f1_score(targets, int_outputs, average = 'micro')
accuracy_scores = accuracy_score(targets, int_outputs)

In [39]:
print("Test Loss:", valid_loss)
print("Hamming Loss:", bert_ham_loss)
print("Jaccard Score:", bert_jacc_score)
print("Label Ranking Average Precision Score:", bert_lrap)
print("F1 Macro Score:", bert_f1_macro)
print("F1 Micro Score:", bert_f1_micro)
print("acc Score:", accuracy_scores)

Test Loss: 0.3775623864624649
Hamming Loss: 0.14281818181818182
Jaccard Score: 0.49894999999999995
Label Ranking Average Precision Score: 0.7510845538720546
F1 Macro Score: 0.5010336549884022
F1 Micro Score: 0.5694710879693067
acc Score: 0.232
